In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta

In [3]:
df = yf.download('TSLA', start ='2021-11-22', interval='15m')

[*********************100%***********************]  1 of 1 completed


In [5]:
df['EMA200'] = ta.trend.ema_indicator(df.Close, window=200)

Datetime
2021-11-22 09:30:00-05:00            NaN
2021-11-22 09:45:00-05:00    1193.670044
2021-11-22 10:00:00-05:00    1194.660034
2021-11-22 10:15:00-05:00    1199.750000
2021-11-22 10:30:00-05:00            NaN
Name: High, dtype: float64

In [9]:
df['wf_Top_bool'] = np.where(
df['High'] == df['High'].rolling(9, center=True).max(), True, False)

In [11]:
df['wf_Top'] = np.where(
df['High'] == df['High'].rolling(9, center=True).max(), df['High'], None)

In [13]:
df['wf_Top'] = df['wf_Top'].ffill()

In [14]:
df.dropna(inplace=True)

In [15]:
df['Buy'] = np.where((df.Close > df.wf_Top) & (df.Close > df.EMA200), 1, 0)

In [16]:
df['SL'] = np.where(df.Buy ==1, df.Close - (df.Close - df.Low), 0)

In [17]:
df['TP'] = np.where(df.Buy ==1, df.Close + (df.Close - df.Low) * 1.5, 0)

In [22]:
buys, sells = [],[]

for i in range(len(df)):
    if df.Buy.iloc[i]:
        buys.append(df.iloc[i].name)
        for j in range(len(df) - i):
            if df.TP.iloc[i] < df.Close.iloc[i + j] or \
            df.SL.iloc[i] > df.Close.iloc[i + j]:
                sells.append(df.iloc[i + j].name)
                break

In [25]:
frame = pd.DataFrame([buys, sells]).T.dropna()

In [27]:
frame.columns = ['Buys', 'Sells']

In [32]:
actualtrades = frame[frame.Buys > frame.Sells.shift(1)]

In [33]:
actualtrades

,Buys,Sells
1,2021-12-22 09:30:00-05:00,2021-12-23 10:00:00-05:00
15,2021-12-27 09:30:00-05:00,2021-12-27 12:00:00-05:00


In [34]:
profits = (df.loc[actualstrades.Sells].Close.values - \
           df.loc[actualstrades.Buys].Close.values) / df.loc[actualtrades.Buys].Close.values

In [35]:
profits

array([0.03684388, 0.02166766])